In [ ]:
from datasets import load_dataset
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, make_scorer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Loading open-food dataset from HuggingFace:

In [ ]:
ds = load_dataset("HC-85/open-food-facts", "nutrition-feats-only")

In [ ]:
df = pd.DataFrame(ds["train"])
df.sample(5, random_state=42)

In [ ]:
df.info()

Displaying the count of missing values in each column to assess the extent of the problem:

In [ ]:
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values[missing_values > 0])

Dropping 'glycemic-index_100g' column as it contains mostly NaN values

In [ ]:
df_dropped = df.drop("glycemic-index_100g", axis=1)
df_dropped

In [ ]:
df_dropped.describe()

In [ ]:
df_dropped.loc[df_dropped["energy_100g"].argmax()].values

Negative values?

# EDA

Preparing separate index variables for text cols and numeric cols:

In [ ]:
text_columns = df_dropped.select_dtypes(include=["object"]).columns
numeric_columns = df_dropped.select_dtypes(include=["number"]).columns
text_columns, numeric_columns

## Correlation

Calculating and visualizing the correlation matrix to identify relationships between variables

In [ ]:
df_corr = df_dropped[numeric_columns].corr()
df_corr

In [ ]:
plt.figure(figsize=(30, 16))
sns.heatmap(df_corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

Identifying pairs of columns with a correlation of 1, indicating perfect redundancy and dropping one of each pair to avoid multicollinearity and reduce dimensionality.

In [ ]:
corr_matrix = df_corr.abs()  # -1 is bad too
upper_tri = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)  # matrix is symmetric

threshold = 0.999

# Find columns with a correlation of 1
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
to_drop

In [ ]:
reduced_df = df_dropped.drop(columns=to_drop)
numeric_columns = reduced_df.select_dtypes(include=["number"]).columns

plt.figure(figsize=(30, 16))
sns.heatmap(reduced_df[numeric_columns].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

# Dropping negative values
Dropping rows with negative values in columns where negatives are not valid. That ensures data integrity and removes potential noise or errors.

In [ ]:
for col in numeric_columns:
    print(col)
    reduced_df = reduced_df[(reduced_df[col] >= 0) | (reduced_df[col].isnull())]
    print(len(reduced_df))

In [ ]:
reduced_df.describe()

# Preprocessing
This will involve handling text features, numeric features, and any necessary transformations.

## Text features
This will involve handling text features, numeric features, and any necessary transformations.

In [ ]:
data = df_dropped
text_columns = data.select_dtypes(include=["object"]).columns
numeric_columns = data.select_dtypes(include=["number"]).columns


s = SimpleImputer(
    missing_values=None, strategy="constant", fill_value=""
).fit_transform(data.loc[data[text_columns].isnull().any(axis=1).index][text_columns])
s

In [ ]:
s_pd = pd.DataFrame(s, columns=text_columns)
s_pd

In [ ]:
CountVectorizer().fit_transform(s_pd["product_name"])

In [ ]:
CountVectorizer().fit_transform(s_pd["quantity"])

In [ ]:
s_pd[s_pd.isnull().any(axis=1)]

## Numeric features
Handling numeric features, which might involve scaling, normalizing, or imputing missing values

In [ ]:
data[numeric_columns]

In [ ]:
knn = KNNImputer(missing_values=np.nan, n_neighbors=3).fit_transform(
    data[numeric_columns]
)
knn

# Making pipeline
Creating a machine learning pipeline that includes preprocessing steps and the model itself

In [ ]:
def pipeline(data: pd.DataFrame, model, param_grid, verbose=100):
    text_columns = data.select_dtypes(include=["object"]).columns
    numeric_columns = data.select_dtypes(include=["number"]).columns

    # Define transformers for categorical and numerical features
    numerical_transformer = Pipeline(
        steps=[
            (
                "imputer",
                KNNImputer(missing_values=np.nan, n_neighbors=2),
            ),  # Impute missing values
            ("scaler", StandardScaler()),  # Standardize features
        ]
    )

    def preprocess_text(data):
        # Ensure data is a string, handle missing values
        return data.astype(str)

    text_transformer = Pipeline(
        steps=[
            (
                "simputer",
                SimpleImputer(missing_values=None, strategy="constant", fill_value=""),
            ),
            ("function", FunctionTransformer(preprocess_text)),
            ("vectorizer", CountVectorizer()),  # Encode text features
            ("scaler", StandardScaler()),
        ]
    )

    # Combine transformers using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, numeric_columns),
            ("text", text_transformer, text_columns),
        ]
    )

    pipe = Pipeline(
        steps=[("preprocessor", preprocessor), ("clustering", model)], verbose=True
    )
    max_score = -1.0

    def silhouette_scorer(estimator, X):
        global max_score
        # Obtain the feature-transformed data
        X_transformed = estimator.named_steps["preprocessor"].transform(X)
        # Predict the cluster labels
        cluster_labels = estimator.named_steps["clustering"].fit_predict(X_transformed)
        # Calculate the silhouette score
        max_score = np.max(silhouette_score(X_transformed, cluster_labels), max_score)
        return max_score

    # Set up GridSearchCV

    grid_search = GridSearchCV(
        pipe, param_grid, cv=5, scoring=silhouette_scorer, n_jobs=-1, verbose=verbose
    )

    # Fit GridSearchCV
    grid_search.fit(data)

    # Get the best parameters and best score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    return best_params, best_score


param_grid = {
    "preprocessor__num__imputer__n_neighbors": [
        1,
        3,
    ],  # Tune n_neighbors for KNNImputer
    "clustering__n_clusters": [4],  # Tune number of clusters for KMeans
    "clustering__init": ["k-means++"],  # Different initializations
    "clustering__max_iter": [500, 1000],  # Max iterations for convergence
}
pipeline(df_dropped[numeric_columns], KMeans(random_state=42), param_grid)